# MyST-NB Usage

The (probably) most useful function of this library is the `myst_nb_metadata_injector`.
This allows you to write _magic-comments_ inside of your Jupyter code cells to inject metadata into those cells and configure the execution/rendering behavior of the [MyST-NB](https://myst-nb.readthedocs.io/en/latest/index.html) library.

For example, with a simple comment like `# remove-input` you can hide the code cell in your rendered documentation and only show the output without having to fight with setting the metadata tags of your code cell.

The main motivation for this functionality is to keep the configuration _local_ and not hide it behind an `metadata` window inside of the Jupyter Notebook UI.
This is especially frustrating for users that use UIs that do not support configuring the code cell metadata, like VSCode.

The inspiration for defining these options _in-line_ as comments comes from the awesome [nbdev](https://github.com/fastai/nbdev) library that takes a very different approach and uses notebooks as the source for _everything_. They generate the source code, documentation, and tests from notebooks.
It is quite an interesting approach, that I would suggest everyone to talk a look at 😃.

## Enabling for MyST-NB

To enable [Myst-NB](https://myst-nb.readthedocs.io/en/latest/index.html) to use the metadata injection preprocessor by default, you have to add the following code to your [conf.py](https://www.sphinx-doc.org/en/master/usage/configuration.html):


```{literalinclude} conf.py
---
language: python
start-after: enable-start
end-before: enable-end
---
```

:::{important}
To overwrite the default `ipynb` extension parser, you are required to use `MyST-NB>=0.14.0`!
At the time of writing, the [jupyter-book project](https://jupyterbook.org/en/stable/intro.html) (`v0.13`) fixes the `MyST-NB` version to `0.13.1` and _cannot_ overwrite the default `ipynb` format.

You can rename your notebooks with a different extension (only characters after the last `.` are considered the extension) and use that option for the custom parser instead.
The main issue with that approach is that some tools have a hard time working with a different extension for Jupyter notebooks (like VSCode).
:::


## Tag configuration

At the time of writing, the following configuration keywords can be set in the `tags` metadata entry of a Jupyter code-cell:

- [](remove-input)
- [](remove-stderr)
- [](remove-output)
- [](remove-cell)
- [](skip-execution)
- [](raises-exception)

For more details, see the `Cell tags` section of the {external+mystnb:doc}`MyST configuration documentation <configuration>`.

To show the _effects_ of the different options, the article will be structured as follows:

1. The jupyter-notebook cell's source code is shown. The visual indicator of the _source_ code of the code cell, is that the stylized code-cell has _no_ border around it.
2. The jupyter-notebook cell's source code is displayed _after_ applying the preprocessor. This will remove the magic comment and inject the string into the metadata field (not shown here). The processed code cell is stylized with a white border and a green left-border. This is the rendered code cell you would be seeing in your documentation.
3. The output of the executed code-cell is shown with a white bordered cell.

And one last visual example to clarify the different _stages_:

Source:
```python
# I am stage 1 and show the 'source' code of
# the code cell
```

Rendered:

In [ ]:
# magic-comment
"""
I am stage 2 and show the post-processed code cell.
If the magic comment matches one of the MyST-NB tags
it would be removed from this cell and injected into the
metadata field.
"""
print("I am the rendered code cell output")

### no-match example

Example showing what happens with code cells that do _not_ include magic comments/are _normal_ code cells.

Source:
```python
# unknown
print("Example showing what happens if the cell doesn't include a magic comment")
```

Rendered:

In [ ]:
# unknown
print("Example showing what happens if the cell doesn't include a magic comment")

Not so surprisingly, nothing much happens with those code-cells. They are simply executed and rendered as-is.

(remove-input)=
### remove-input

Hides the source code of the cell.
Useful if you are only interested in the output.

Source:
```python
# remove-input
print("My code cell is hidden")
```

Rendered:

In [ ]:
# remove-input
print("My code cell is hidden")

(remove-stderr)=
### remove-stderr

Only remove the standard error output of the code cell.
This is useful if an underlying library uses `stderr` to print progress or other debug messages that would only pollute the output for the documentation.

Source:
```python
# remove-stderr
import sys
print("The stderr output is removed", file=sys.stderr)
print("But not the default output")
```

Rendered:

In [ ]:
# remove-stderr
import sys

print("The stderr output is removed", file=sys.stderr)
print("But not the default output")

(remove-output)=
### remove-output

Remove the output of the code cell (duh).
Useful to hide some warning during important without having filter the messages or to hide visual output.

Source:
```python
# remove-output
import sys
print("My output is removed")
print("including the error output", file=sys.stderr)
```

Rendered:

In [ ]:
# remove-output
import sys

print("My output is removed")
print("including the error output", file=sys.stderr)

(remove-cell)=
### remove-cell

Hide the input as well as the output of the executed cell.

Source:
```python
# remove-cell
print("This entire cell is removed")
```

Rendered:

In [ ]:
# remove-cell
print("This entire cell is removed")

As you can (can't ?) see the cell is missing from the documentation page.

```{note}
The code-cell is still executed under the hood!
```

(raises-exception)=
### raises-exception

The following code raises an exception and has 
`raises-exception` set to signal that the exception  is expected and continues with the execution and build process of the documentation.
This is helpful, if you are showing examples of _failing_ code and want to include the trace-back.

Source:
```python
# raises-exception
1 / 0
```

Rendered:

In [ ]:
# raises-exception
1 / 0

(skip-execution)=
### skip-execution

The following code _would_ raise an exception but the execution of the cell is skipped and not exception is raised.


Source:
```python
# skip-execution
1 / 0
```

Rendered:

In [ ]:
# skip-execution
1 / 0

Notice how no output is produced and no exception is raised.

## Cell conf

In [ ]:
# remove_code_source = true

print("My code source should be removed")